In [4]:
import string

def remove_punctuation(input_string):
    # Make a translation table that maps all punctuation characters to None
    translator = str.maketrans("", "", string.punctuation)

    # Apply the translation table to the input string
    result = input_string.translate(translator)

    return result

In [5]:
import pandas as pd

papers = pd.read_csv("../data/arxiv_papers_full_v2.csv")
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558 entries, 0 to 3557
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3558 non-null   int64 
 1   Title             3558 non-null   object
 2   PDF URL           3558 non-null   object
 3   Author            3558 non-null   object
 4   DOI               403 non-null    object
 5   Published Date    3558 non-null   object
 6   Summary           3558 non-null   object
 7   Journal Ref       515 non-null    object
 8   Primary Category  3558 non-null   object
 9   Category          3558 non-null   object
 10  Entry ID          3558 non-null   object
dtypes: int64(1), object(10)
memory usage: 305.9+ KB


In [6]:
from PyPDF2 import PdfReader
from PyPDF2.errors import PdfReadError

def get_full_text(pdf_url):
    
    pdf_folder = '../PDF/'
    filename = pdf_url.split("/")[-1]
    full_file_name = pdf_folder + filename + ".pdf"
    
    try:
        reader = PdfReader(full_file_name)
        extracted_text =""
        for page in reader.pages:
            extracted_text += page.extract_text()
    except PdfReadError:
        return "Read Error"
    except:
        return "Error"
        
    return extracted_text


In [7]:

from tqdm import tqdm
tqdm.pandas()

papers["Full Text"] = papers.progress_apply(lambda x : get_full_text(x["PDF URL"]), axis=1)

papers.info()

  7%|▋         | 266/3558 [07:35<33:04,  1.66it/s]  c:\Users\dmccaskill\source\repos\SGHacks\SGHacks\venv\lib\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
 15%|█▌        | 539/3558 [11:17<35:13,  1.43it/s]  Illegal character in Name Object (b'/ABCDEE+\xb7s\xb2\xd3\xa9\xfa\xc5\xe9')
Illegal character in Name Object (b'/ABCDEE+\xb7s\xb2\xd3\xa9\xfa\xc5\xe9')
 21%|██▏       | 761/3558 [15:43<26:00,  1.79it/s]  FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
FloatObject (b'0.000000000000-2842171') invalid; use 0.0 instead
FloatObject (b'0.000000000000-5684342') invalid; use 0.0 instead
 42%|████▏     | 1486/3558 [27:40<1:16:52,  2.23s/it] unknown widths : 
[0, IndirectObject(162, 0, 1873694812192)]
 59%|█████▉    | 2112/3558 [37:37<11:02,  2.18it/s]  FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 inst

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558 entries, 0 to 3557
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        3558 non-null   int64 
 1   Title             3558 non-null   object
 2   PDF URL           3558 non-null   object
 3   Author            3558 non-null   object
 4   DOI               403 non-null    object
 5   Published Date    3558 non-null   object
 6   Summary           3558 non-null   object
 7   Journal Ref       515 non-null    object
 8   Primary Category  3558 non-null   object
 9   Category          3558 non-null   object
 10  Entry ID          3558 non-null   object
 11  Full Text         3558 non-null   object
dtypes: int64(1), object(11)
memory usage: 333.7+ KB


In [8]:
papers.to_csv("../data/arxiv_papers_full_v2_w_full_text.csv")

In [23]:
import pandas as pd

papers = pd.read_csv("../data/arxiv_papers_full_v2_w_full_text.csv")
papers.head(1)


,Unnamed: 0.1,Unnamed: 0,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID,Full Text
0,0,0,Natural Language Processing using Hadoop and K...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"Natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,Natural Language Processing using Hadoop and ...


In [12]:

import re
import wordninja 


# def seperate_long_words(text):
#     words = text.split()
#     for word in words:
#         if len(word) > 10:
#             word = wordninja.split(word)


def process_text(text):
    # Remove numbers
    text = ' '.join(word for word in text.split() if not word.isdigit())  

    # Remove new line
    text = text.replace("/n", " ")

    # Normalize text by removing extra spaces
    text = ' '.join(text.split())

    text = text.lower()

    return text


In [13]:

papers["Full Text"] = papers.progress_apply(lambda x : process_text(x["Full Text"]), axis=1)
papers["Summary"] = papers.progress_apply(lambda x : process_text(x["Summary"]), axis=1)
papers["Title"] = papers.progress_apply(lambda x : process_text(x["Title"]), axis=1)

papers.head()

100%|██████████| 3558/3558 [00:00<00:00, 55529.72it/s]


,Unnamed: 0,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID,Full Text
0,0,natural language processing using hadoop and k...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,natural language processing using hadoop and k...
1,1,integrating ai planning with natural language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,natural language processing (nlp) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,integrating ai planning with natural language ...
2,2,simple natural language processing tools for d...,http://arxiv.org/pdf/1906.11608v2,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,this technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,arxiv:1906.11608v2 [cs.cl] jul 2019simple natu...
3,3,towards the study of morphological processing ...,http://arxiv.org/pdf/2006.16212v1,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,there is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,arxiv:2006.16212v1 [cs.cl] jun 2020towards the...
4,4,natural language understanding with distribute...,http://arxiv.org/pdf/1511.07916v1,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,this is a lecture note for the course ds-ga <n...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,natural language understanding with distribute...


In [16]:
def return_id_string(title):

    subset = papers[papers["Full Text"].str.contains(title, regex=False)]
    indices = subset.index.tolist()
    return indices

In [17]:
papers["Citations"] = papers.progress_apply(lambda x : return_id_string(x["Title"]), axis=1)

papers["Citation Count"] = papers["Citations"].apply(len)

papers.to_csv("../data/arxiv_papers_full_v2_w_full_text_and_cite.csv")

papers["Citation Count"].max()

100%|██████████| 3558/3558 [03:50<00:00, 15.46it/s]


314

In [18]:
papers.head()

,Unnamed: 0,Title,PDF URL,Author,DOI,Published Date,Summary,Journal Ref,Primary Category,Category,Entry ID,Full Text,Citations,Citation Count
0,0,natural language processing using hadoop and k...,http://arxiv.org/pdf/1608.04434v1,"[arxiv.Result.Author('Emre Erturk'), arxiv.Res...",NaN,2016-08-15 23:09:21+00:00,"natural language processing, as a data analyti...",NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1608.04434v1,natural language processing using hadoop and k...,[0],1
1,1,integrating ai planning with natural language ...,http://arxiv.org/pdf/2202.07138v2,"[arxiv.Result.Author('Kebing Jin'), arxiv.Resu...",NaN,2022-02-15 02:19:09+00:00,natural language processing (nlp) aims at inve...,NaN,cs.AI,"['cs.AI', 'cs.CL']",http://arxiv.org/abs/2202.07138v2,integrating ai planning with natural language ...,[1],1
2,2,simple natural language processing tools for d...,http://arxiv.org/pdf/1906.11608v2,[arxiv.Result.Author('Leon Derczynski')],NaN,2019-06-27 13:15:12+00:00,this technical note describes a set of baselin...,NaN,cs.CL,['cs.CL'],http://arxiv.org/abs/1906.11608v2,arxiv:1906.11608v2 [cs.cl] jul 2019simple natu...,[2],1
3,3,towards the study of morphological processing ...,http://arxiv.org/pdf/2006.16212v1,"[arxiv.Result.Author('Mirinso Shadang'), arxiv...",NaN,2020-06-29 17:24:09+00:00,there is no or little work on natural language...,In proceeding of Regional International Confer...,cs.CL,['cs.CL'],http://arxiv.org/abs/2006.16212v1,arxiv:2006.16212v1 [cs.cl] jun 2020towards the...,[],0
4,4,natural language understanding with distribute...,http://arxiv.org/pdf/1511.07916v1,[arxiv.Result.Author('Kyunghyun Cho')],NaN,2015-11-24 23:23:13+00:00,this is a lecture note for the course ds-ga <n...,NaN,cs.CL,"['cs.CL', 'stat.ML']",http://arxiv.org/abs/1511.07916v1,natural language understanding with distribute...,[4],1
